In [53]:
import numpy as np
import math
import matplotlib.pyplot as plt

from Library import unit_of_measure as um
from Library import NIST330 as un
from Library import Water as h2o

umm = un.Length.mm
uin = un.Length.inch
usqmm = um.Unit('mm^2', un.Area.meterSq.Dimension, 0.000001)
upsi = un.Pressure.psi
ulbf = un.Force.lbf


In [54]:
# Reference ASME B1.1-2003 Appendix B for the thread strength design formulas.

class ThreadSeriesUN:

    def __init__(self, size, tpi, series, cls):
        self._angle = 30
        self._d = size
        self._P = 1.0 / tpi
        self._series = series
        self._cls = cls
        self.__calculate_basic()
        self.__calculate_std_LE()
        self.__calculate_limits()

    def __calculate_basic(self):
        # These calculations are for UN thread profile only.  The UNR profile is not covered.

        # start by calculating the dimensions of the basic profile
        self._D = self._d                                       # Major diameter, internal thread
        self._H = math.cos(math.radians(self._angle)) * self._P # 10.1(d): Height of sharp V-thread (fundamental triangle)
        self._hs = 5.0 / 8.0 * self._H                          # 10.1(f): Height of external thread
        self._hn = self._hs                                     # 10.1(f): Height of internal thread

        # flat widths are expressed as function of pitch
        self._Fcs = 1.0 / 8.0 * self._P   # 10.1(h): Flat at crest of external thread
        self._Frs = 1.0 / 4.0 * self._P   # 10.1(h): Flat at root of external thread
        self._Fcn = self._Frs             # 10.1(k): Flat at crest of internal thread
        self._Frn =  1.0 / 8.0 * self._P  # 10.1(m): Flat at root of internal thread

        # truncations are expressed as functions of thread height
        self._fcs = 1.0 / 8.0 * self._H # 10.1(i): Truncation at crest of external tread
        self._fcn = 1.0 / 4.0 * self._H # 10.1(l): Truncation at crest of internal tread
        self._frn = 1.0 / 8.0 * self._H # 10.1(n): Truncation at root of internal tread
        self._frs = self._fcn           #          Truncation at root of external tread

        self._has = 3.0 / 8.0 * self._H  # 10.1(o): Addendum of external thread (pitch to crest of basic thread)
        self._hds = self._hs - self._has #          Dedendum of external thread (pitch to root of basic thread)
        self._hdn = self._has            #          Dedendum of internal thread (pitch to root of basic thread)
        self._han = self._hn - self._hdn #          Addendum of external thread (pitch to crest of basic thread)
        self._hb  = 2.0 * self._has      #          Table 5, column 13

        self._d2 = self._d - 2.0 * self._has # 10.1(p): Pitch diameter of external thread.
        self._d1 = self._d - 2.0 * self._hs  # 10.1(r): Minor diameter of UN external thread.
        self._D2 = self._D - 2.0 * self._hdn #          Pitch diameter of internal thread.
        self._D1 = self._D - 2.0 * self._hn  # 10.1(s): Minor diameter of internal thread.

        return self

    def __calculate_std_LE(self):
        # Section 5.1, Pitch Diameter Tolerance, All Classes
        self._LE = 0.0
        if self._series in [ 'UNC', 'UNF', '4-UN', '6-UN', '8-UN' ]:
            self._LE = self._D
        elif self._series in [ 'UNEF', '12-UN', '16-UN', '20-UN', '28-UN', '32-UN']:
            self._LE = 9.0 * self._P
        elif 'UNS' in self._series:
            self._LE = 9.0 * self._P
        elif 'Special' in self._series:
            self._LE = self._LE # Length of thread engagement must already be set.
        else:
            raise Exception(f'thread series "{self._series}" is not recognized')

        if self._cls not in [1, 2, 3]:
            raise Exception(f'thread class "{self._cls}" is not recognized.  Must be 1, 2, or 3')

        return self

    def __calculate_limits(self):
        # 5.8.1(b): Major diameter tolerance (External Threads)
        self._d_tol = 0.060 * math.pow(self._P, 2.0 / 3.0) # class 2 and 3
        if self._cls == 1:
            self._d_tol = 0.090 * math.pow(self._P, 2.0 / 3.0)

        # 5.8.1(c): Pitch diameter tolerance (External Threads)
        self._d2_tol = 0.0015 * (math.pow(self._D, 1.0 / 3.0) + math.sqrt(self._LE) + 10.0 * math.pow(self._P, 2.0 / 3.0)) # class 2
        if self._cls == 1:
            self._d2_tol *= 1.500
        elif self._cls == 3:
            self._d2_tol *= 0.750

        # 5.8.2(b): Pitch diameter tolerance (Internal Threads)
        self._D2_tol = 1.300 * self._d2_tol

        # 5.8.2(a): Major diameter tolerance (Internal Threads)
        self._D_tol = 1.0 / 6.0 * self._H + self._D2_tol

        # 5.8.2(c): Minor diameter tolerance (Internal Threads)
        D1_tol_A = 0.050 * math.pow(self._P, 2.0 / 3.0) + 0.030 * self._P / self._D - 0.002
        D1_tol_B = 0.394 * self._P
        D1_tol_C = 0.250 * self._P - 0.400 * self._P ** 2
        D1_tol_D = 0.150 * self._P
        D1_tol_E = 0.230 * self._P - 1.500 * self._P ** 2
        D1_tol_F = 0.120 * self._P

         # 5.8.2(c)(1) - not considering length of engagement because no alternate formulas were given.
        if self._cls in [1, 2]:
            if self._D < 0.250:
                self._D1_tol = D1_tol_A
                if self._D1_tol > D1_tol_B: # 5.8.2(c)(1)(a)
                    self._D1_tol = D1_tol_B
                elif self._D1_tol < D1_tol_C: # 5.8.2(c)(1)(b)
                    self._D1_tol = D1_tol_C
            else: # size is .250 and larger
                self._D1_tol = D1_tol_C
        else: # class 3 thread  # 5.8.2(c)(2)
            self._D1_tol = D1_tol_A
            if self._D1_tol > D1_tol_B: # 5.8.2(c)(2)(a)
                self._D1_tol = D1_tol_B
            else:
                if self._P < 0.08: # TPI >= 13 
                    if self._D1_tol < D1_tol_E: # 5.8.2(c)(2)(b)(1)
                        self._D1_tol = D1_tol_E
                else:
                    if self._D1_tol < D1_tol_F: # 5.8.2(c)(2)(b)(2)
                        self._D1_tol = D1_tol_F

        # 5.8.1(a): Allowance (applies to external threads only)
        self._allowance = 0.300 * self._d2_tol # class 1 and 2
        if self._cls == 3:
            self._allowance = 0.0

        # Table 17A
        self._d_max = self._d                              # Maximum major diameter
        self._d1_max = self._D1                            # Minor diameter, reference only 8.3.1(e)(1)(b)
        if self._cls in [1, 2]:
            self._d_max -= self._allowance
            self._d1_max -= self._allowance                # Maximum minor diameter, reference only 8.3.1(e)(1)(a)

        self._d_min = self._d_max - self._d_tol            # Minimum major diameter
        self._d2_max = self._d_max - self._hb              # Maximum pitch diameter
        self._d2_min = self._d2_max - self._d2_tol         # Minimum pitch diameter
        self._d1_min = self._d2_min - 0.64951905 * self._P # Minimum minor diameter, reference only 8.3.1(f)

        # Table 17B
        self._D1_min = self._D - 2.0 * self._hn            # Minimum minor diameter
        self._D1_max = self._D1_min + self._D1_tol          # Maximum minor diameter
        self._D2_min = self._D -  self._hb                 # Minimum pitch diameter
        self._D2_max = self._D2_min + self._D2_tol         # Maximum pitch diameter
        self._D_min = self._D                              # Minimum major diameter, 8.3.2(b), reference
        self._D_max = self._D_min + self._D_tol            # Maximum major diameter, 5.8.2(a)(1)

        return self

    def __str__(self):
        return f'size: {self._d} pitch: {self._P} pitch dia: {self._d2:0.4f} minor dia: {self._d1:0.4f}'

    @property
    def IncludedAngle(self):
        return (self._angle * 2.0)

    @property
    def HalfAngle(self):
        return self._angle

    @property
    def Size(self):
        return self._d

    @property
    def Pitch(self):
        return self._pitch

    @property
    def TPI(self):
        return 1.0 / self._pitch

    @property
    def Series(self):
        return self._series

    @property
    def Class(self):
        return self._cls

    @property
    def LE(self):
        return self._LE

    @LE.setter
    def LE(self, le):
        self._LE = le
        return self.__calculate_limits()

    def BasicDims(self):
        return f'major Φ: {self._d:0.4f}  pitch Φ: {self._d2:0.4f}  minor Φ: {self._d1:0.4f}  pitch: {self._P:0.4f}  hn: {self._hn:0.4f}  tsa: {self.TensileStressArea():0.5f}'

    def Major_External(self):
        nom = (self._d_max + self._d_min) / 2.0
        return np.round([self._d_max, nom, self._d_min], 4)

    def Pitch_External(self):
        nom = (self._d2_max + self._d2_min) / 2.0
        return np.round([self._d2_max, nom, self._d2_min], 4)

    def Minor_External(self):
        nom = (self._d1_max + self._d1_min) / 2.0
        return np.round([self._d1_max, nom, self._d1_min], 4)

    def Major_Internal(self):
        nom = (self._D_max + self._D_min) / 2.0
        return np.round([self._D_max, nom, self._D_min], 4)

    def Pitch_Internal(self):
        nom = (self._D2_max + self._D2_min) / 2.0
        return np.round([self._D2_max, nom, self._D2_min], 4)

    def Minor_Internal(self):
        nom = (self._D1_max + self._D1_min) / 2.0
        return np.round([self._D1_max, nom, self._D1_min], 4)

    def TensileArea(self):
        tsa = math.pi * ((self._D2 / 2.0 - 3.0 / 16.0 * self._H) ** 2)
        return tsa

    def ShearAreaInternal(self):
        partA = math.pi * self._LE * self._d_min
        partB = self._P / 2.0 + 2.0 / 3.0 * math.cos(math.radians(self._angle))*(self._d_min - self._D2_max)
        return partA * partB

    def ShearAreaExternal(self):
        partA = math.pi * self._LE * self._D1_max
        partB = self._P / 2.0 + 2.0 / 3.0 * math.cos(math.radians(self._angle))*(self._d2_min - self._D1_max)
        return partA * partB

d = 0.5
tpi = 28
t1 = ThreadSeriesUN(d, tpi, 'UNEF', 2)
print(f'LE: {t1.LE:0.4f}   allow: {t1._allowance:0.4f}   p tol: {t1._d2_tol:0.6f}   TSA: {t1.TensileArea():0.6f}')
print(t1.BasicDims())
print(f'Ext - Maj: {t1.Major_External()}  Ext - Pitch: {t1.Pitch_External()}  Ext - Minor: {t1.Minor_External()}')
print(f'Int - Maj: {t1.Major_Internal()}  Int - Pitch: {t1.Pitch_Internal()}  Int - Minor: {t1.Minor_Internal()}')


    



def BasicPitchDia(D, _pitch):
    H = _pitch * np.cos(30.0 / 180.0)
    D2 = D - H * 0.75

    return D2

def _AreaTensileStress(D2, _pitch):
    H = _pitch * np.cos(30.0 / 180.0)
    x = D2 / 2.0
    x += H * 3.0 / 16.0
    y = x.squared() * np.pi

    return y

def AreaTensileStress(D, _pitch):
    x = D
    x -= _pitch * 1.125
    y = x.squared() * (np.pi / 4.0)

    return y

def AreaShearInternal(D2max, dmin, _pitch, LE):
    x = LE * dmin / _pitch * np.pi
    y = _pitch / 2.0
    y +=  (dmin - D2max) * 2.0 / 3.0 * np.cos(30.0 / 180.0)
    z = x * y
 
    return z

def AreaShearExternal(D1max, d2min, _pitch, LE):
    x = LE * D1max / _pitch * np.pi
    y = _pitch / 2.0
    y +=  (d2min - D1max) * 2.0 / 3.0 * np.cos(30.0 / 180.0)
    z = x * y
 
    return z


LE: 0.3214   allow: 0.0011   p tol: 0.003668   TSA: 0.169972
major Φ: 0.5000  pitch Φ: 0.4768  minor Φ: 0.4613  pitch: 0.0357  hn: 0.0193  tsa: 0.16997
Ext - Maj: [0.4989 0.4956 0.4924]  Ext - Pitch: [0.4757 0.4739 0.472 ]  Ext - Minor: [0.4602 0.4545 0.4488]
Int - Maj: [0.5099 0.505  0.5   ]  Int - Pitch: [0.4816 0.4792 0.4768]  Int - Minor: [0.4698 0.4655 0.4613]


In [55]:
dCoverTube = 1.812 * uin
aCoverTube = dCoverTube.squared() * np.pi / 4.0
PMO = 100.0 * upsi

load = aCoverTube * PMO
print(f"load due to oil pressure: {load.ValueAsStr(ulbf)}")
print()

# oil manifold - post thread
# Machinery's Handbook, 24th Ed.  Tables 11 and 12.  Pages 1584 and 1587
D     = 18.0 * umm     # basic major diameter
P     = 1.50 * umm     # thread pitch
D1max = 16.676 * umm   # internal maximum minor diameter
D2max =  17.216 * umm  # internal maximum pitch diameter
dmin  =  17.732 * umm  # external minimum major diameter
d2min = 16.854 * umm   # external minimum pitch diameter
LE    = 0.379 * uin    # length of thread engagement

print('Oil manifold - post threaded joint')
print('-Oil manifold internal thread')

ASint = AreaShearInternal(D2max, dmin, P, LE)
print (f'--thread shear area: {ASint.ValueAsStr(usqmm)}')
stress = load / ASint
print(f'--thread shear stress: {stress.ValueAsStr(upsi)}')

print('-Post external thread')
At = AreaTensileStress(D, P)
print(f"--tensile stress area: {At.ValueAsStr(usqmm)}")
stress = load / At
print(f'--thread tensile stress: {stress.ValueAsStr(upsi)}')
ASext = AreaShearExternal(D1max, d2min, P, LE)
print (f'--thread shear area: {ASext.ValueAsStr(usqmm)}')
stress = load / ASext
print(f'--thread shear stress: {stress.ValueAsStr(upsi)}')


# cap - post thread 
# Machinery's Handbook, 24th Ed.  Tables 11 and 12.  Pages 1584 and 1587
D     = 16.0 * umm     # basic major diameter
P     = 1.50 * umm     # thread pitch
D1max = 14.676 * umm   # internal maximum minor diameter
D2max =  15.216 * umm  # internal maximum pitch diameter
dmin  =  15.732 * umm  # external minimum major diameter
d2min = 14.854 * umm   # external minimum pitch diameter
LE    = 0.379 * uin    # length of thread engagement

print()
print('Tubular membrane cap - post threaded joint')
print('-Tubular membrane cap internal thread')

ASint = AreaShearInternal(D2max, dmin, P, LE)
print (f'--thread shear area: {ASint.ValueAsStr(usqmm)}')
stress = load / ASint
print(f'--thread shear stress: {stress.ValueAsStr(upsi)}')

print('-Post external thread')
At = AreaTensileStress(D, P)
print(f"--tensile stress area: {At.ValueAsStr(usqmm)}")
stress = load / At
print(f'--thread tensile stress: {stress.ValueAsStr(upsi)}')
ASext = AreaShearExternal(D1max, d2min, P, LE)
print (f'--thread shear area: {ASext.ValueAsStr(usqmm)}')
stress = load / ASext
print(f'--thread shear stress: {stress.ValueAsStr(upsi)}')



load due to oil pressure: 257.8732347402644 lbf

Oil manifold - post threaded joint
-Oil manifold internal thread
--thread shear area: 389.41274528534416 mm^2
--thread shear stress: 427.2317692197808 psi
-Post external thread
--tensile stress area: 208.99261050311554 mm^2
--thread tensile stress: 796.0544429034218 psi
--thread shear area: 291.51016921527486 mm^2
--thread shear stress: 570.7159258726534 psi

Tubular membrane cap - post threaded joint
-Tubular membrane cap internal thread
--thread shear area: 345.4907122055625 mm^2
--thread shear stress: 481.5454952838262 psi
-Post external thread
--tensile stress area: 160.88697299502184 mm^2
--thread tensile stress: 1034.076861711714 psi
--thread shear area: 256.54852742884185 mm^2
--thread shear stress: 648.4913314153981 psi
